In [46]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import os
import sys

# Add parent directory to Python path
sys.path.append('..')

from models import CustomCNN, VGG16, ViT

print("✅ Dependencies loaded. Using device:", "GPU" if torch.cuda.is_available() else "CPU")


✅ Dependencies loaded. Using device: CPU


In [47]:
# Cell 2: Function to load and preprocess an image
def load_image(image_path):
    """
    Load and preprocess an image to 64x64, normalized like in training.
    """
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")

    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # [-1, 1]
    ])

    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension
    return img_tensor

In [48]:
# Cell 3: Initialize and load trained models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model instances (same architecture as Étape 1)
models_dict = {
    "Custom CNN": CustomCNN(),
    "VGG16": VGG16(),
    "ViT-1": ViT(n_blocks=1),
    "ViT-2": ViT(n_blocks=2)
}

# Paths to saved weights
model_paths = {
    "Custom CNN": "../etape1/models/CustomCNN.pth",
    "VGG16": "../etape1/models/VGG16.pth",
    "ViT-1": "../etape1/models/ViT-1.pth",  # Changed from ViT_1.pth
    "ViT-2": "../etape1/models/ViT-2.pth"   # Changed from ViT_2.pth
}

# Load state dicts
for name, model in models_dict.items():
    if not os.path.exists(model_paths[name]):
        raise FileNotFoundError(f"Missing model file: {model_paths[name]}")
    model.load_state_dict(torch.load(model_paths[name], map_location=device, weights_only=True))
    model.to(device)
    model.eval()  # Set to evaluation mode

print(":-) All 4 models loaded successfully.")

:-) All 4 models loaded successfully.


In [49]:
# Cell 4: Run prediction on a given image path
def predict_image(image_path):
    print(f"\n==> Analyzing: {os.path.basename(image_path)}\n")
    print("-" * 50)

    # Load and move image to device
    img = load_image(image_path).to(device)

    # Predict with each model
    for name, model in models_dict.items():
        with torch.no_grad():
            outputs = model(img)
            probabilities = torch.softmax(outputs, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()

        label = "Malade" if predicted_class == 1 else "Saine"
        print(f"{name:15} → {label} (confiance: {confidence:.3f})")

    print("-" * 50)

In [50]:
# Cell 5: Example usage
# Replace this path with any image you want to test
test_image_path = "test_images/ISIC_0024306_0.jpg"  # ← put a test image in your folder
#test_image_path = "test_images/ISIC_0024313_1.jpg"

# Optional: upload an image if running on Colab
# from google.colab import files
# uploaded = files.upload()
# test_image_path = list(uploaded.keys())[0]

# Run prediction
try:
    predict_image(test_image_path)
except Exception as e:
    print("Error:", e)
    print("\n Tip: Place an image (e.g., 'test.jpg') in this folder and update the path above.")


==> Analyzing: ISIC_0024306_0.jpg

--------------------------------------------------
Custom CNN      → Saine (confiance: 0.919)
VGG16           → Saine (confiance: 0.926)
ViT-1           → Saine (confiance: 1.000)
ViT-2           → Saine (confiance: 1.000)
--------------------------------------------------


In [51]:
# Cell 6 (Optional): Predict multiple images
test_folder = "test_images/"  # create this folder with images

if os.path.exists(test_folder):
    for img_name in os.listdir(test_folder):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            predict_image(os.path.join(test_folder, img_name))
else:
    print(" No 'test_images' folder found. Skip batch prediction.")


==> Analyzing: ISIC_0024306_0.jpg

--------------------------------------------------
Custom CNN      → Saine (confiance: 0.919)
VGG16           → Saine (confiance: 0.926)
ViT-1           → Saine (confiance: 1.000)
ViT-2           → Saine (confiance: 1.000)
--------------------------------------------------

==> Analyzing: ISIC_0024313_1.jpg

--------------------------------------------------
Custom CNN      → Saine (confiance: 0.892)
VGG16           → Saine (confiance: 0.926)
ViT-1           → Saine (confiance: 0.971)
ViT-2           → Saine (confiance: 0.945)
--------------------------------------------------


In [52]:
import os

models_dir = "../etape1/models"
if os.path.exists(models_dir):
    print("Available model files:")
    for f in sorted(os.listdir(models_dir)):
        if f.endswith('.pth'):
            print(f"  - {f}")
else:
    print(f"Directory not found: {models_dir}")

Available model files:
  - CustomCNN.pth
  - VGG16.pth
  - ViT-1.pth
  - ViT-2.pth
